In [2]:
# %load ../../scripts/fold_enrichment.py
import pandas as pd
import numpy as np
import argparse
from collections import defaultdict
from scipy.stats import linregress

In [18]:
#####   Parse Input   #####
out_path = './'
drop_samples = [line.strip() for line in open('./drop.txt', 'r')] ## J dropped because all noisy. S @ 20ug is fine, but all others are 2ug. Set aside for now.
control_col = 'input'

# pd.DataFrame(index=oligoID, columns=sampleID, values=proportions of reads in each column assigned to each oligo)
proportions = pd.read_csv('./proportions.csv', index_col=0)
alt_samples = [c for c in proportions.columns if '20ug' in c] ## Work with the 2ug samples, set aside the 20ug samples for now

# drop bad replicates or superfluous samples as specified
proportions.drop(drop_samples+alt_samples, inplace=True, axis=1, errors='ignore')

# all columns with 'input' in the name
input_cols = [c for c in proportions.columns.values if 'input' in c.lower()]
# all columns with 'beads' in the name
beads_cols = [c for c in proportions.columns.values if 'beads' in c.lower()]

# nonnumerical columns
metadata_cols = ['virus', 'start', 'end', 'sequence', 'start', 'end', 'strains']
metadata = proportions[metadata_cols]

# all non-metadata columns
proportions = proportions[[c for c in proportions.columns.values if c not in metadata_cols]]

In [19]:
#####   Aggregate replicates    #####
def aggregate(df, reps, name):
    if len(reps) == 1:
        df[name] = df[reps[0]]
    else:
        df[name] = df[reps].mean(axis=1) # mean of each row --> new column of aggregated values
    df.drop(reps, inplace=True, axis=1, errors='ignore') # drop original columns (yes, the axis designator switches between these two methods which is incredibly annoying but is correct I promise)

technical_replicates = defaultdict(list)
# {'NHP-3-2ng': ['NHP-3-2ng-1', 'NHP-3-2ng-2']}

for serum in proportions.columns.values: # Find replicates like ['NHP-3-1', 'NHP-3-2']
    if serum in beads_cols:
        name = 'beads'
    elif serum in input_cols:
        name = 'input'
    else:
        name = serum.rsplit('_', 2)[0]
    technical_replicates[name].append(serum)

for serum, reps in technical_replicates.items():
    aggregate(proportions, reps, serum)
# now data looks like pd.DataFrame(columns=['NHP-1', 'input', 'beads', 'NHP-2', ...])
print proportions.head()

                DENV1_30dpi       EVU  DENV3_30dpi  DENV4_30dpi         H  \
id                                                                          
1                  0.005368  0.000346     0.000350     0.000340  0.000225   
100                0.000035  0.000006     0.000072     0.000122  0.000059   
1001               0.000033  0.000023     0.000083     0.000131  0.000000   
1002               0.000183  0.000719     0.000072     0.000102  0.000497   
1008.1177.1346     0.000917  0.000662     0.001098     0.000690  0.002028   

                   input         A         C         B         E    ...     \
id                                                                  ...      
1               0.001005  0.000849  0.000299  0.001010  0.000953    ...      
100             0.000230  0.000104  0.000195  0.000204  0.000086    ...      
1001            0.000165  0.000144  0.000149  0.000137  0.000155    ...      
1002            0.000107  0.000317  0.000186  0.000242  0.000391    ..

In [20]:
#####   Convert all values to fold enrichment over control  #####
def fold_enrichment(row):
    ctrl = row[control_col]
    def enrichment_value(i):
        if ctrl == 0.:
            ctrl = proportions[control_col].mean() # for the rare 0 values in the control, plug in the mean control value
        return float(i) / float(ctrl)
    return row.map(enrichment_value)

mean_control_val = proportions[control_col].mean() # avoid dividing by 0
filled_control_col = proportions[control_col].replace(0., mean_control_val)
enrichment = proportions.divide(filled_control_col, axis=0)

annotated_enrichment = enrichment.join(metadata) # reattach the metadata
print annotated_enrichment.head()
annotated_enrichment.to_csv(out_path+'enrichment.csv') # write to file

                DENV1_30dpi       EVU  DENV3_30dpi  DENV4_30dpi         H  \
id                                                                          
1                  5.342450  0.344297     0.348807     0.338078  0.223641   
100                0.153189  0.024564     0.313813     0.530782  0.257395   
1001               0.202503  0.142185     0.503929     0.791249  0.000000   
1002               1.717832  6.749681     0.677535     0.960523  4.661363   
1008.1177.1346     0.401863  0.290015     0.481123     0.302460  0.888652   

                input         A         C         B         E  \
id                                                              
1                 1.0  0.844821  0.297357  1.005429  0.949002   
100               1.0  0.451624  0.847315  0.888415  0.373681   
1001              1.0  0.870367  0.900856  0.827343  0.937925   
1002              1.0  2.970106  1.747152  2.268064  3.668605   
1008.1177.1346    1.0  0.813334  0.632322  0.608222  0.503840   

    